In [36]:
# Import delle librerie

import numpy as np
import os
import string
import random
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import ops
from tensorflow.keras.layers import TextVectorization

os.environ["KERAS_BACKEND"] = "tensorflow"

In [37]:
# Esempio di stringhe SMILES (in un'applicazione reale, useresti un dataset più ampio)
smiles_corpus = [
    "CC(C)CC1=CC=CC=C1", "CC(C)C1=CC=C(C=C1)C(O)=O", "CC(C)C1=CC=CC=C1O", "CCC1=CC=C(C=C1)C=O",
    "CCOC(=O)C1=CC=CC=C1", "CC(C)C1=CC=C(C=C1)O", "COC1=CC=C(C=C1)C(O)=O", "CCCC1=CC=C(C=C1)O",
    "CC(C)CC1=CC=CC=C1O", "CC(C)COC1=CC=CC=C1", "CCC1=CC=CC=C1OC", "CCOC1=CC=CC=C1C",
    "CCCC1=CC=CC=C1C", "CC(C)C1=CC=C(C=C1)C=O", "COC1=CC=C(C=C1)C=O", "CCC1=CC=C(C=C1)C=O",
    "CC(C)C1=CC=C(C=C1)C(O)=O", "CCOC1=CC=CC=C1O", "CCCC1=CC=C(C=C1)O", "CC(C)C1=CC=CC=C1C",
    "CCCC1=CC=CC=C1C=O", "CCOC1=CC=CC=C1C=O", "CC(C)COC1=CC=CC=C1O", "COC1=CC=C(C=C1)C(O)=O",
    "CC(C)C1=CC=CC=C1OC", "CCCC1=CC=CC=C1OC", "COC1=CC=CC=C1OC", "CCC1=CC=C(C=C1)C(O)=O",
    "CCCC1=CC=CC=C1OC=O", "CCOC1=CC=CC=C1OC", "CC(C)C1=CC=CC=C1CO", "CCCC1=CC=C(C=C1)C=O",
    "CCOC1=CC=CC=C1CO", "COC1=CC=CC=C1CO", "CC(C)CC1=CC=C(C=C1)O", "CC(C)CC1=CC=CC=C1CO",
    "CCCC1=CC=CC=C1CO", "CCOC1=CC=CC=C1COC", "COC1=CC=C(C=C1)CO", "CCCC1=CC=CC=C1COC",
    "CC(C)C1=CC=CC=C1COC", "CCCC1=CC=C(C=C1)COC", "COC1=CC=C(C=C1)COC", "CCOC1=CC=CC=C1COC",
    "CC(C)C1=CC=C(C=C1)C=O", "CCCC1=CC=CC=C1COC=O", "CC(C)CC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1COC=O",
    "COC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1C(O)=O", "CC(C)COC1=CC=CC=C1C(O)=O",
    "CC(C)C1=CC=C(C=C1)C=O", "CCCC1=CC=C(C=C1)COC=O", "CCOC1=CC=CC=C1C(O)=O", "COC1=CC=C(C=C1)COC=O",
    "CCCC1=CC=CC=C1C(O)=O", "CC(C)C1=CC=CC=C1COC=O", "COC1=CC=CC=C1C(O)=O", "CCCC1=CC=CC=C1C(O)=O",
    "CCOC1=CC=CC=C1C=O", "CC(C)C1=CC=CC=C1COC", "COC1=CC=C(C=C1)COC=O", "CCCC1=CC=CC=C1COC=O",
    "CC(C)C1=CC=C(C=C1)COC=O", "CCCC1=CC=CC=C1COC=O", "COC1=CC=C(C=C1)COC", "CCOC1=CC=CC=C1COC=O",
    "CC(C)COC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC=O", "CC(C)COC1=CC=CC=C1COC", "COC1=CC=CC=C1COC",
    "CCOC1=CC=CC=C1COC=O", "CC(C)C1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC", "CC(C)COC1=CC=CC=C1COC",
    "COC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC=O", "COC1=CC=CC=C1COC",
    "CC(C)COC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1COC=O",
    "CC(C)COC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC",
    "CCOC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC=O",
    "CCOC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC",
    "COC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1COC=O", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC=O",
    "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC",
    "COC1=CC=CC=C1COC=O", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC",
    "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC",
    "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC",
    "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC",
    "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC",
    "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC",
    "CCCC1=CC=CC=C1COC", "COC1=CC=CC=C1COC", "CCOC1=CC=CC=C1COC", "CCCC1=CC=CC=C1COC"
]

path = "smiles_files/"

for i, smile in enumerate(smiles_corpus):
    filename = path + f"smile_{i+1}.txt"  # Creating a unique filename for each string
    with open(filename, "w") as file:
        file.write(smile)

filenames = []
directories = [
    "smiles_files"
]
for dir in directories:
    for f in os.listdir(dir):
        filenames.append(os.path.join(dir, f))

In [38]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 80  # Max sequence size
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer

# Create a dataset from text files
random.shuffle(filenames)
text_ds = tf_data.TextLineDataset(filenames)
text_ds = text_ds.shuffle(buffer_size=10)
text_ds = text_ds.batch(batch_size=2)

def custom_standardization(input_text):
    # Convert the input text to lowercase (if you want case insensitivity)
    return tf.strings.regex_replace(input_text, "", " ")

# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    max_tokens=vocab_size - 1,
    output_sequence_length=maxlen + 1,
    output_mode="int",
    standardize=custom_standardization,  # Use the custom split function
)

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()  # To get words back from token indices

# print the vocabulary
print(vocab)

def prepare_lm_inputs_labels(text):
    """
    Shift word sequences by 1 position so that the target for position (i) is
    word at position (i+1). The model will use all words up till position (i)
    to predict the next word.
    """
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


text_ds = text_ds.map(prepare_lm_inputs_labels, num_parallel_calls=tf_data.AUTOTUNE)
text_ds = text_ds.prefetch(tf_data.AUTOTUNE)

['', '[UNK]', 'C', '=', '1', 'O', ')', '(']


In [39]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = ops.arange(n_dest)[:, None]
    j = ops.arange(n_src)
    m = i >= j - n_src + n_dest
    mask = ops.cast(m, dtype)
    mask = ops.reshape(mask, [1, n_dest, n_src])
    mult = ops.concatenate(
        [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])], 0
    )
    return ops.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, "bool")
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [40]:
class TextGenerator(tf.keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = ops.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(ops.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x, verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")

In [41]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(0, maxlen, 1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [42]:
def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype="int32")
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam",
        loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model

In [43]:
# Tokenize starting prompt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

start_prompt = "C"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 10
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)


In [44]:
model = create_model()

model.fit(text_ds, verbose=2, epochs=5, callbacks=[text_gen_callback])

Epoch 1/5


C:\Users\AndreaBellome\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


KeyboardInterrupt: 